# Calibration parameter extractor

Extracts calibration matrix (with parameters) for scale, biases and skew, misaligment.

## Gyroscope

In [ ]:
import numpy as np
import pandas as pd


data_train = pd.read_csv("data/train.csv")
data_calib = pd.read_csv("data/calib.csv")

data_train['GyroX'] *= 0.001 * (np.pi / 180)
data_train['GyroY'] *= 0.001 * (np.pi / 180)
data_train['GyroZ'] *= 0.001 * (np.pi / 180)

g_bias_x = np.mean(data_train['GyroX'][:1000])
g_bias_y = np.mean(data_train['GyroY'][:1000])
g_bias_z = np.mean(data_train['GyroZ'][:1000])

gyro_vars = np.var(data_train[['GyroX', 'GyroY', 'GyroZ']][:1000], axis=0)

print(g_bias_x, g_bias_y, g_bias_z)
print(gyro_vars)

0.0012691044958167846 -0.010023475459808865 -0.004176814457081565
GyroX    7.280557e-07
GyroY    3.816278e-07
GyroZ    3.455532e-07
dtype: float64


## Accelerometer

Calibration model:
$$
\underbrace{
\begin{bmatrix}
x_{\text{calibrated}} \\
y_{\text{calibrated}} \\
z_{\text{calibrated}}
\end{bmatrix}
}_{b_{calibrated}}
=
\underbrace{
\begin{bmatrix}
a & b & c \\
d & e & f \\
g & h & i
\end{bmatrix}
}_{C_{params}}
\underbrace{
\begin{bmatrix}
x_{\text{raw}} \\
y_{\text{raw}} \\
z_{\text{raw}}
\end{bmatrix}
}_{b_{raw}}
+
\underbrace{
\begin{bmatrix}
j \\
k \\
l
\end{bmatrix}
}_{b_{biases}}
$$

...rewritten (method for getting calibrated data from measurements):
$$
\underbrace{
\begin{bmatrix}
x_{\text{calibrated}} & y_{\text{calibrated}} & z_{\text{calibrated}}
\end{bmatrix}
}_{b_{calibrated}}
=
\underbrace{
\begin{bmatrix}
x_{\text{raw}} & y_{\text{raw}} & z_{\text{raw}} & 1
\end{bmatrix}
}_{b_{raw} \& 1}
\underbrace{
\begin{bmatrix}
a & d & g \\
b & e & h \\
c & f & i \\
j & k & l
\end{bmatrix}
}_{X = C_{params} \& b_{biases}}
$$



$$x_{true} := x_{calibrated}$$

$$
R = \begin{bmatrix}
x_{\text{raw}1} & y_{\text{raw}1} & z_{\text{raw}1} & 1 \\
x_{\text{raw}2} & y_{\text{raw}2} & z_{\text{raw}2} & 1 \\
\vdots & \vdots & \vdots & \vdots \\
x_{\text{raw}N} & y_{\text{raw}N} & z_{\text{raw}N} & 1
\end{bmatrix}
$$

$$
T = \begin{bmatrix}
x_{\text{true}1} & y_{\text{true}1} & z_{\text{true}1} \\
x_{\text{true}2} & y_{\text{true}2} & z_{\text{true}2} \\
\vdots & \vdots & \vdots \\
x_{\text{true}N} & y_{\text{true}N} & z_{\text{true}N}
\end{bmatrix}
$$


$$T = R \cdot X$$
$$R \cdot X = T$$
Finally, to calculate parameter matrix $X$:
$$X = \left(R^T \cdot R\right)^{-1} \cdot R^T \cdot T$$

In [4]:
data_calib.head()

,Id,Time,AccX,AccY,AccZ,GyroX,GyroY,GyroZ,MagX,MagY,MagZ,Temp,Humi
0,1,0.000,-0.024108,-0.054566,0.991768,178.8416,-578.0264,-195.2574,-0.134030,0.613407,-0.051774,24.47058,54.89778
1,2,0.005,-0.026156,-0.056275,0.996317,219.3900,-542.0506,-184.3858,-0.133570,0.615318,-0.058530,24.45736,54.90110
2,3,0.010,-0.023688,-0.054196,0.991651,188.0404,-587.2462,-233.0581,-0.138280,0.615764,-0.058878,24.44750,54.91940
3,4,0.015,-0.026729,-0.055779,0.998567,197.6931,-558.0608,-264.9692,-0.144074,0.618320,-0.054028,24.44116,54.95213
4,5,0.020,-0.022881,-0.053497,0.995859,127.9740,-554.9653,-245.5636,-0.144719,0.619271,-0.052017,24.43837,54.99829


In [37]:
import numpy as np
import pandas as pd
from ekf_filter import GRAVITY

data_train = pd.read_csv("data/train.csv")
data_calib = pd.read_csv("data/calib.csv")

sin45deg = np.sin(np.pi/4)  # sin(45deg)
cos45deg = np.cos(np.pi/4)  # cos(45deg)

# Dane z akcelerometru w g -> m/s^2
# data_train['AccX'] *= GRAVITY
# data_train['AccY'] *= GRAVITY
# data_train['AccZ'] *= GRAVITY
# data_calib['AccX'] *= GRAVITY
# data_calib['AccY'] *= GRAVITY
# data_calib['AccZ'] *= GRAVITY

# Measurements for calibration
# xup_x  , xup_y  , xup_z   =  1.2,  0.0,  0.1
# xdown_x, xdown_y, xdown_z = -1.0,  0.0,  0.1
# yup_x  , yup_y  , yup_z   =  0.1,  1.1,  0.1
# ydown_x, ydown_y, ydown_z =  0.1, -1.1,  0.1
# zup_x  , zup_y  , zup_z   =  0.1,  0.0,  1.1
# zdown_x, zdown_y, zdown_z =  0.1,  0.0, -0.9

# raw = np.array([
#     [xup_x  , xup_y  , xup_z  , 1],
#     [xdown_x, xdown_y, xdown_z, 1],
#     [yup_x  , yup_y  , yup_z  , 1],
#     [ydown_x, ydown_y, ydown_z, 1],
#     [zup_x  , zup_y  , zup_z  , 1],
#     [zdown_x, zdown_y, zdown_z, 1]
# ])

# true_matrix = np.array([
#     [ 1,  0,  0],
#     [-1,  0,  0],
#     [ 0,  1,  0],
#     [ 0, -1,  0],
#     [ 0,  0,  1],
#     [ 0,  0, -1]
# ])

data_ranges = [
    {'src': data_calib, 'from': 0,     'to': 350,   'x': 0,     'y': 0,         'z': 1},
    {'src': data_calib, 'from': 1150,  'to': 1280,  'x': 1,     'y': 0,         'z': 0},
    {'src': data_calib, 'from': 3400,  'to': 3500,  'x': -1,    'y': 0,         'z': 0},
    {'src': data_calib, 'from': 4500,  'to': 4650,  'x': 0,     'y': 0,         'z': -1},
    {'src': data_calib, 'from': 14900, 'to': 15200, 'x': 0,     'y': 1,         'z': 0},
    {'src': data_train, 'from': 4050,  'to': 4300,  'x': 0,     'y': sin45deg,  'z': cos45deg},
]

raw_matrix = []
true_matrix = []

for data_range in data_ranges:
    idx_from = data_range['from']
    idx_to = data_range['to']
    
    rawX = np.mean(data_range['src']['AccX'][idx_from:idx_to])
    rawY = np.mean(data_range['src']['AccY'][idx_from:idx_to])
    rawZ = np.mean(data_range['src']['AccZ'][idx_from:idx_to])

    # print(rawX, rawY, rawZ)
    # print(data_range['x'], data_range['y'], data_range['z'])

    raw_matrix.append([rawX, rawY, rawZ, 1])
    # true_matrix.append(np.array([data_range['x'], data_range['y'], data_range['z']]) * GRAVITY)
    true_matrix.append(np.array([data_range['x'], data_range['y'], data_range['z']]))

raw_matrix = np.array(raw_matrix)
true_matrix = np.array(true_matrix)

X = np.linalg.inv(raw_matrix.T @ raw_matrix) @ raw_matrix.T @ true_matrix

# Calibrate raw values
# raw = np.array([
#     [xup_x, xup_y, xup_z, 1]
# ])

calibrated = raw_matrix @ X
print(X)
print(calibrated)

[[ 9.88464657e-01 -5.19657834e-03 -3.96220374e-03]
 [-1.54499968e-02  1.01390834e+00 -4.16419325e-02]
 [ 1.26581711e-02  4.26682476e-02  1.00297773e+00]
 [-2.84017972e-04  2.23789880e-02 -4.27369506e-03]]
[[-0.01192825  0.00878199  1.00074498]
 [ 1.00579967 -0.00997641  0.00238658]
 [-0.99376984 -0.01018539  0.00230769]
 [-0.00373642  0.01087959 -1.00368222]
 [-0.00806174  0.99737403  0.00462746]
 [ 0.01169657  0.71023296  0.70072229]]


In [21]:
data_train = pd.read_csv("data/train.csv")

# Dane z akcelerometru w g -> m/s^2
data_train['AccX'] *= GRAVITY
data_train['AccY'] *= GRAVITY
data_train['AccZ'] *= GRAVITY

# ACC Variances
acc_vars = np.var(data_train[['AccX', 'AccY', 'AccZ']][:1000], axis=0)

acc_bias_x = np.mean(data_train['AccX'][:1000])
acc_bias_y = np.mean(data_train['AccY'][:1000])
acc_bias_z = np.mean(data_train['AccZ'][:1000]) - GRAVITY

print(acc_vars)

print(acc_bias_x, acc_bias_y, acc_bias_z)

AccX    0.000388
AccY    0.000115
AccZ    0.000789
dtype: float64
-0.03625491076241041 -0.4961249187201 -0.016753359078000685


## Magnetometer